In [1]:
import numpy as np
from Strategy import Player

In [2]:
class Deck():
    def __init__(self,):
        self.cards = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']
        self.face_cards = ['J','Q','K']
    def draw(self,):
        card_drawn = np.random.choice(self.cards)
        return card_drawn

In [3]:
class BlackJack():
    def __init__(self, player):
        self.deck = Deck()
        self.hands_to_play = []
        self.dealer_public = None
        self.dealer_hidden = None
        self.bet = None
        self.hands_to_resolve = []
        self.player = player      
        self.bankrolls = []
    def deal(self, ):
        player_hand = [self.deck.draw(), self.deck.draw()]
        self.dealer_public = self.deck.draw()
        self.dealer_hidden = self.deck.draw()
        self.hands_to_play.append(player_hand)
        
    def reset(self, bet):
        self.bet = bet
        self.player.bankroll -= self.bet
        self.deal()
        return (self.hands_to_play, self.hands_to_resolve, self.dealer_public)
    
    def hand_to_value(self, hand):
        value = 0
        aces = 0
        for card in hand:
            if card in self.deck.face_cards:
                value = value + 10
            elif card == 'A':
                value = value + 11
                aces = aces + 1
            else:
                value = value + int(card)
        while (value > 21 and aces > 0):
            value = value - 10
            aces = aces - 1
        return value
    
    
    def dealer_hit_strategy(self, dealer_hand):
        dealer_score = self.hand_to_value(dealer_hand)
        
        if(dealer_score < 17):
            dealer_hand.append(self.deck.draw())
            return self.dealer_hit_strategy(dealer_hand)

        elif(dealer_score in range(17, 22)):
            return dealer_score, self.check_blackjack(dealer_hand)

        elif(dealer_score > 21):
            return 0, self.check_blackjack(dealer_hand)
    
    def check_blackjack(self, hand):
        value = self.hand_to_value(hand)
        if value == 21 and len(hand) == 2:
            return 1
        else:
            return 0
            
    def resolve(self, player_hand, dealer_score, dealer_has_blackjack):
        
        player_score = self.hand_to_value(player_hand)
        player_has_blackjack = self.check_blackjack(player_hand)
        
        if(self.check_player_bust(player_hand)):
            return 0
        
        if(dealer_has_blackjack):
            if(player_has_blackjack):
                return 1
            else:
                return 0
        
        if(player_score > dealer_score):
            if player_has_blackjack:
                return 2.5
            else:
                return 2
        
        if(player_score == dealer_score):
            return 1
        
        if(player_score < dealer_score):
            return 0
           
    
    def check_player_bust(self, hand):
        value = self.hand_to_value(hand)
        if value > 21:
            return True
        else:
            return False

    def step_hand(self, action, player_hand):
        print(action, player_hand)
        player_has_blackjack = self.check_blackjack(player_hand)
        
        if player_has_blackjack:
            return player_hand, 1
        
        else:
            if action == 'stand':
                return player_hand, 1
            
            elif action == 'hit':
                player_hand.append(self.deck.draw())
                is_player_bust = self.check_player_bust(player_hand)
                if is_player_bust:
                    return player_hand, 1
                else:
                    return player_hand, 0
                
            elif action == 'double':
                self.bet = self.bet * 2
                self.player.bankroll -= self.bet
                player_hand.append(self.deck.draw())
                is_player_bust = self.check_player_bust(player_hand)
                if is_player_bust:
                    return player_hand, 1
                else:
                    self.hands_to_resolve.append(player_hand)
                    return player_hand, 1
            
            elif action == 'split':
                assert len(player_hand) == 2, 'Player has more than two cards'
                assert player_hand[0] == player_hand[1], 'Player`s cards are different'
                
                self.player.bankroll -= self.bet
                self.hands_to_play.append([player_hand[0], self.deck.draw()])
                player_hand = [player_hand[0], self.deck.draw()]
                return player_hand, 0
            
    def step(self):
        while(len(self.hands_to_play) > 0):
            for player_hand in self.hands_to_play:
                done = 0
                while not done:
                    action = self.player.choose_action(player_hand, self.dealer_public)
                    player_hand, done = self.step_hand(action, player_hand)
                    
            self.hands_to_resolve.append(player_hand)
            self.hands_to_play.remove(player_hand)
            
        dealer_score, dealer_has_blackjack = self.dealer_hit_strategy([self.dealer_public, self.dealer_hidden])
        
        for player_hand in self.hands_to_resolve:
            reward = self.resolve(player_hand, dealer_score, dealer_has_blackjack)
            self.player.bankroll += reward * self.bet
            self.bankrolls.append(self.player.bankroll)
        

In [4]:
env = BlackJack(Player(30))
env.reset(2)

([['8', 'J']], [], '7')

In [5]:
for i in range(200):
    env.step()
    env.reset(2)

stand ['8', 'J']
stand ['4', 'J']
double ['4', '7']
hit ['3', '2']
hit ['3', '2', '10']
double ['3', '8']
hit ['2', 'Q']
stand ['2', 'Q', '7']
hit ['2', '2']
hit ['2', '2', '2']
hit ['2', '2', '2', '2']
hit ['2', '2', '2', '2', '5']
stand ['K', '10']
stand ['10', '3']
stand ['7', 'Q']
hit ['K', '4']

 21
stand ['A', 'Q']
split ['8', '8']
hit ['8', '6']
stand ['8', '9']
split ['8', '8']
hit ['8', '7']
hit ['8', '4']
hit ['8', '4', '3']
stand ['8', '4', '3', '6']
split ['8', '8']
stand ['8', 'Q']
stand ['8', 'J']
split ['8', '8']
stand ['8', 'J']
double ['8', '3']
split ['8', '8']
split ['8', '8']
stand ['8', 'K']

 19
stand ['8', 'A']
stand ['8', 'J']
split ['8', '8']
stand ['8', '10']

 19
stand ['8', 'A']
hit ['8', '4']
split ['8', '8']
stand ['8', 'K']

 19
stand ['8', 'A']
stand ['8', '9']
split ['8', '8']
stand ['8', 'J']

 19
stand ['8', 'A']
stand ['8', 'J']
split ['8', '8']
hit ['8', '6']

 19
stand ['8', 'A']
hit ['8', '2']
hit ['8', '2', '3']
split ['8', '8']
hit ['8', '2']
hi

stand ['8', 'K']
double ['8', '3']
hit ['8', '5']
stand ['8', '5', '5']
split ['8', '8']
hit ['8', '2']
stand ['8', '2', 'K']
double ['8', '3']

 19
stand ['8', 'A']
hit ['8', '6']
hit ['8', '6', '2']
hit ['8', '7']
hit ['8', '7']
stand ['8', 'J']
split ['8', '8']
hit ['8', '2']
stand ['8', '2', 'J']
split ['8', '8']
stand ['8', 'J']
split ['8', '8']
double ['8', '3']
stand ['8', '6', '3']
stand ['8', '3', '3', '4']
stand ['8', '9']
stand ['8', '3', '4', '4']
stand ['8', '5', '4']

 19
stand ['8', 'A']
stand ['8', '2', '4', 'J']
stand ['8', '5', '10']
stand ['8', '4', '7']
stand ['8', '10']
stand ['8', '10']
stand ['8', 'Q']
stand ['8', 'Q']
stand ['8', '6', 'Q']
stand ['8', '3', '7']
stand ['8', '10']

 19
stand ['8', 'A']
split ['8', '8']
stand ['8', '10']
stand ['8', '10']
split ['8', '8']
hit ['8', '5']

 24
stand ['8', '6', 'A', '9']
stand ['8', '10']

 21
stand ['8', '2', 'A']

 19
stand ['8', 'A']
stand ['8', 'J']
stand ['8', '4', '9']
stand ['8', 'Q']
stand ['8', '4', '6']

 19

stand ['8', '9']
stand ['8', '3', 'J']
stand ['8', 'K']
stand ['8', '9']
stand ['8', '5', '7']
split ['8', '8']
double ['8', '3']
stand ['8', '5', 'K']
stand ['8', 'J']
stand ['8', 'Q']
stand ['8', 'K']
stand ['8', 'K']
stand ['8', 'Q']
stand ['8', '4', '5']
stand ['8', '9']
stand ['8', '2', 'Q']

 24
stand ['8', '5', 'A', '10']
stand ['8', '2', 'Q']
stand ['8', '7', '9']

 19
stand ['8', 'A']
stand ['8', '9']
stand ['8', '10']

 19
stand ['8', 'A']
split ['8', '8']
hit ['8', '5']
stand ['8', '5', '7']

 19
stand ['8', 'A']
stand ['8', '10']
stand ['8', '5', 'K']
split ['8', '8']
stand ['8', '9']
stand ['8', '5', '5']
stand ['8', 'Q']
stand ['8', '6', '8']
stand ['8', 'J']
stand ['8', '3', 'K']
stand ['8', 'J']
stand ['8', '2', 'Q']
stand ['8', '10']
stand ['8', '6', '7']

 19
stand ['8', 'A']
hit ['8', '3', '4']
stand ['8', '9']
stand ['8', '7', 'K']
stand ['8', 'K']
split ['8', '8']
split ['8', '8']
hit ['8', '4']
hit ['8', '4', '4']
stand ['8', 'K']
stand ['8', 'K']
stand ['8', '10'

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
env.step()